In [1]:
import spacy
mod = spacy.load("en_core_web_sm")

In [2]:
import pandas as pd 

In [3]:
df = pd.read_csv(r"C:\Users\Divya\skills\machineL\deep_learning\dataset\cleaned_data.csv")

In [4]:
def extract_aspects(review):
    document = mod(review)
    pairs = []

    for word in document:
        if word.dep_ == "amod" and word.head.pos_ == "NOUN":
            aspect = word.head.lemma_.lower()
            opinion = word.lemma_.lower()
            pairs.append((aspect,opinion))

        if word.dep_ == "nsubj" and word.head.pos_ == "ADJ":
            aspect = word.lemma_.lower()
            opinion = word.head.lemma_.lower()
            pairs.append((aspect,opinion))

    return list(pairs)

In [6]:
df = df.dropna(subset=["review"])

In [7]:
df["aspects"] = df["review"].apply(extract_aspects)

In [12]:
gen_words = {"this","thing","stuff","product","day","you","that","they","movie","who","what","which","them","year","all","something","tha","time","way","the","him","her","man","nothing","bit", "kaffi","thi","fact","a lot","anything","world","thi","kaffi",
             "some","problem","guy","anyone","review","point","this one","one","people","she","lot","part","hour","fast","slow","movie","product","place","order","experience","issue","app","this movie","this book","the movie","the film","end","the book","book","film","diya",
             "kar","the end","minute","kaffi","love","bilkul","this film","line","woman","reason","raha","kaffi","girl","money",""}

def filter_asp(aspect_list):
    return [a for a in aspect_list if a not in gen_words]

df["aspects"] = df["aspects"].apply(filter_asp)

In [16]:
from collections import Counter 

all_aspect = []
for aspect in df["aspects"]:
    all_aspect.extend(aspect)

freq = Counter(all_aspect)
# print(freq.most_common(30))
def rem_rare(aspect_list):
    return [a for a in aspect_list if freq[a]>=10]

df['aspects'] = df['aspects'].apply(rem_rare)

In [17]:
df['aspects']

0        [(movie, whole), (bit, least), (majority, vast...
1        [(thank, nice), (phone, mobile), (delivery, fa...
2                                         [(phone, great)]
3                                          [(phone, good)]
4                                                       []
                               ...                        
99259                                     [(sense, great)]
99260                                    [(product, good)]
99261        [(blue, deep), (player, great), (blue, deep)]
99262                                                   []
99263    [(it, available), (thing, only), (thing, bad),...
Name: aspects, Length: 99191, dtype: object

In [ ]:

df["aspect_count"] = df["aspects"].apply(len)
print(df["aspect_count"].mean())

1.9726084019719532


In [19]:
df.to_pickle("reviews_with_aspects.pkl")